In [1]:
!python3 -V

Python 3.8.15


In [2]:
!pip3 install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
!pip install fastai==2.2.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.4 MB/s eta 0:03:56tcmalloc: large alloc 1147494400 bytes == 0x393f6000 @  0x7f27d6698615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |█████████████████████████████▏  | 1055.7 MB 1.2 MB/s eta 0:01:26tcmalloc: large alloc 1434370048 bytes == 0x7da4c000 @  0x7f27d6698615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |█████

In [3]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from fastai.text.all import *
import gc

In [4]:
print(torch. __version__)

1.7.1+cu110


In [41]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

In [42]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
#Remember to use num_workers=0 when creating the DataBunch.

In [7]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [8]:
!ls /content/drive/MyDrive/YelpDataset/dataframes

AutoDF.pkl		     HealthMedicalDF.pkl  restaurantDF.pkl
beautyandSpaDF.pkl	     HomeServicesDF.pkl   shoppingDF.pkl
EventPlanningServicesDF.pkl  LocalServicesDF.pkl
foodDF.pkl		     NightlifeDF.pkl


In [43]:
%cp /content/drive/MyDrive/YelpDataset/dataframes/HomeServicesDF.pkl /content/

In [44]:
import pandas as pd
df = pd.read_pickle('HomeServicesDF.pkl')

In [45]:
df = df.rename(columns={'text': 'reviewText'})
df = df.rename(columns={'stars': 'overall'})
df = df.rename(columns={'categories': 'category'})

In [46]:
df = df[['business_id','name','city','state','overall','category','reviewText']]

In [47]:
df.head(1)

,business_id,name,city,state,overall,category,reviewText
96964,REwfwz-_-CMQ7Np5UVi9Qg,Jennie Deckert,Carmel,IN,5.0,Home Services,"I've gotta say we couldn't have been more pleased with Jen.\n\nHer patience with us through every step of the process was bar none. And as a first-time homebuyer that's been invaluable. She helped lead us through the fog to understand what we wanted for ourselves, and focus on what was most important for us, providing knowledgeable info on neighborhoods and housing options, despite our wide open search criteria and times of indecisiveness. She even lasted through a deal that fell through after an inspection, with no hint of complaint. \n\nI felt like Jen was Always there for us (over wh..."


In [48]:
df['sentenceLength'] = df['reviewText'].str.count(' ') + 1

In [49]:
df.shape

(166452, 8)

In [50]:
df.head(2)

,business_id,name,city,state,overall,category,reviewText,sentenceLength
96964,REwfwz-_-CMQ7Np5UVi9Qg,Jennie Deckert,Carmel,IN,5.0,Home Services,"I've gotta say we couldn't have been more pleased with Jen.\n\nHer patience with us through every step of the process was bar none. And as a first-time homebuyer that's been invaluable. She helped lead us through the fog to understand what we wanted for ourselves, and focus on what was most important for us, providing knowledgeable info on neighborhoods and housing options, despite our wide open search criteria and times of indecisiveness. She even lasted through a deal that fell through after an inspection, with no hint of complaint. \n\nI felt like Jen was Always there for us (over wh...",173
96965,eMjnw_7wp-CscyNh6Lu0ZA,AM&PM Locksmith,Philadelphia,PA,4.5,Home Services,Quick 24hr effificent locksmith. Charlie was our tech and he was very fast and understanding. The work was done well and quickly. Not the cheapest but we didn't mind paying for same day quick excellent service. We couldn't be more pleased.,41


In [51]:
sample_df = df.sample(frac=0.05, random_state=2021).dropna().reset_index()

In [52]:
sample_df.shape

(8323, 9)

In [53]:
del df
gc.collect()

19

In [54]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [55]:
learn = Learner(dls=None, model=model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

### Generated examples of Toys_and_Games_5

In [56]:
!ls /content/drive/MyDrive/YelpDataset/models

gpt2-finetuned-yelp-reviews-sample-1cycle-Automotive.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-BeautySpa.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Food.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-HomeServices.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Restaurants.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Shopping.pth


In [57]:
%cp /content/drive/MyDrive/YelpDataset/models/gpt2-finetuned-yelp-reviews-sample-1cycle-HomeServices.pth /content/

In [58]:
learn.load("gpt2-finetuned-yelp-reviews-sample-1cycle-HomeServices").cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [59]:
"""N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 10
BINS = list(range(10,15))"""
N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 2000
BINS = list(range(10,351))

In [60]:
import warnings
warnings.filterwarnings(action="ignore")
from sklearn.model_selection import train_test_split
generated_df_rows = []
for i in range(len(BINS)-1):
    sample_row_df = sample_df[(sample_df["sentenceLength"]>=BINS[i]) & (sample_df["sentenceLength"]<BINS[i+1])]
    bin_proportion = (sample_row_df.shape[0])/float(LEN_SAMPLE_DF)
    n_reviews_to_generate = int((bin_proportion)*N_REVIEWS)+1
    try:
        or_df, cg_df =  train_test_split(sample_row_df, test_size=0.5, random_state=2021)
    except Exception as e:
        print(e)
        continue
    sample_reviews_df = cg_df.sample(n=n_reviews_to_generate,random_state=2021)
    or_sample_reviews_df = or_df.sample(n=n_reviews_to_generate,random_state=2021)
    if BINS[i] in range(10,21):
        max_len = 20
    else:
        max_len = BINS[i]
    for index, row in sample_reviews_df.iterrows():
        text = row["reviewText"]
        prompt = (" ").join(text.strip().split(" ")[:N_INITIAL_WORDS])
        prompt_ids = tokenizer.encode(prompt)
        inp = tensor(prompt_ids)[None].cuda()
        # set seed to reproduce results
        random_seed(seed_value=2021, use_cuda=True)
        preds = learn.model.generate(inp, max_length=max_len, do_sample=True, top_k=0, top_p=0.92, temperature=0.7)
        decoded_preds = tokenizer.decode(preds[0],skip_special_tokens=True)
        generated_df_rows.append([row["category"],row["overall"],decoded_preds,"CG"])
    for index, row in or_sample_reviews_df.iterrows():
        generated_df_rows.append([row["category"],row["overall"],row["reviewText"],"OR"])
#     break
generated_df = pd.DataFrame(generated_df_rows, columns=["category", "rating", "text", "label"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [61]:
generated_df.shape

(4096, 4)

In [62]:
generated_df.head()

,category,rating,text,label
0,Home Services,3.0,What a great place to live. The guys at the front desk are always very helpful and helpful.,CG
1,Home Services,4.5,I really love this guy. He has the utmost knowledge and patience. He is the person you,CG
2,Home Services,1.5,Terrible!!! Don't waste your time. The owner does not care!!,OR
3,Home Services,5.0,"Excellent, friendly and very responsive service! Reasonable prices. Highly recommend!",OR
4,Home Services,3.5,Excellent customer service. I appreciate that. The guys at the front desk are also very professional and helpful,CG


In [63]:
def post_process(str, k=3):
    if not str.endswith("."):
        full_stop_split_str = str.split(".")
        clean_full_stop_split_str = [s.strip() for s in full_stop_split_str if s.strip()]
        if len(clean_full_stop_split_str) > 1:
            last_sentence = clean_full_stop_split_str[-1]
        else:
            return str
        last_sentence_len = len(last_sentence.split(" "))
        if last_sentence_len > k:
            return str
        return (". ").join(clean_full_stop_split_str[:-1]).strip()+(".")
    return str

In [64]:
generated_df['text_'] = generated_df['text'].apply(lambda x: post_process(x))

In [65]:
generated_df.head()

,category,rating,text,label,text_
0,Home Services,3.0,What a great place to live. The guys at the front desk are always very helpful and helpful.,CG,What a great place to live. The guys at the front desk are always very helpful and helpful.
1,Home Services,4.5,I really love this guy. He has the utmost knowledge and patience. He is the person you,CG,I really love this guy. He has the utmost knowledge and patience. He is the person you
2,Home Services,1.5,Terrible!!! Don't waste your time. The owner does not care!!,OR,Terrible!!! Don't waste your time. The owner does not care!!
3,Home Services,5.0,"Excellent, friendly and very responsive service! Reasonable prices. Highly recommend!",OR,"Excellent, friendly and very responsive service! Reasonable prices."
4,Home Services,3.5,Excellent customer service. I appreciate that. The guys at the front desk are also very professional and helpful,CG,Excellent customer service. I appreciate that. The guys at the front desk are also very professional and helpful


In [66]:
generated_df.drop(columns=["text"],axis=1,inplace=True)

In [67]:
generated_df[100:112]

,category,rating,label,text_
100,Home Services,2.0,CG,I do not recommend this company. The only reason I did not sign my lease was because they were
101,Home Services,5.0,CG,"Great customer service, fine product, and professional.We just had a horrible experience with the Clipper"
102,Home Services,3.5,CG,Chris was soooo good! He showed me the pieces and explained what he was going to do to
103,Home Services,4.5,CG,Showed up promptly and fixed the problem within an hour. I will definitely recommend him to anyone looking
104,Home Services,4.5,CG,"Wow! After these guys did a great job installing the ceiling fan, I was amazed at how well"
105,Home Services,4.5,CG,"The ladies did an excellent job. The guys were polite, courteous, and were always available"
106,Home Services,5.0,CG,Turn out they were having issues with the system and they were able to come out the next day.
107,Home Services,1.5,CG,Company advertises on local radio stations. The only problem is that they don't have the local channels
108,Home Services,2.5,CG,They just caused a flood and destroyed the entire property. I will not be using this company again.
109,Home Services,2.5,OR,The apartments are adorable and the grounds are always well kept. The staff is very prompt and helpful :)


In [68]:
generated_df.to_csv("Generated_Sentences_HomeServices.csv", index=None)

In [69]:
%cp /content/Generated_Sentences_HomeServices.csv /content/drive/MyDrive/YelpDataset/generatedData